In [1]:
# Authenticate Google Cloud in Colab
from google.colab import auth
auth.authenticate_user()

In [2]:
# Install Required Libraries
!pip install google-cloud-storage google-cloud-bigquery nltk pandas

In [3]:
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025' # Make sure this is the correct bucket name
blob_name = 'datasets/mlb-fan-content-interaction-data/2025-mlb-fan-favs-follows.json'

# Get the blob directly without getting the bucket metadata first
# This assumes the bucket is public and blob_name includes the full path
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file
raw_data = blob.download_as_text()

# Print the data (optional)
print(raw_data[0:100])

{"user_id":"K1M8UWG6EIITHOR","favorite_team_id":"108","followed_player_ids":[],"followed_team_ids":[


In [7]:
# load json with pandas
import pandas as pd
import json
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025'
blob_name = 'datasets/mlb-fan-content-interaction-data/2025-mlb-fan-favs-follows.json'

# Get the blob directly
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file as bytes
data = blob.download_as_bytes()

# Decode the bytes to a string
raw_data = data.decode('utf-8')

# Load the data using json.loads to handle possible JSON Lines format
try:
    # Attempt to load as a single JSON object
    json_data = json.loads(raw_data)
    df = pd.DataFrame([json_data]) # Enclose in a list if it's a single object
except json.JSONDecodeError:
    # If single object fails, try loading as JSON Lines
    json_data = [json.loads(line) for line in raw_data.splitlines() if line.strip()]
    df = pd.DataFrame(json_data)

# Preview data
df.head()

,user_id,favorite_team_id,followed_player_ids,followed_team_ids
0,K1M8UWG6EIITHOR,108,[],"[116, 147, 139, 133]"
1,1LOQWJRV7HJYY9P,108,[],"[147, 138]"
2,HCFH38U843NWUO2,108,[],"[135, 117, 143, 119, 141, 115]"
3,8JYOELC88CWR9DP,108,[],"[141, 109, 112, 115, 119]"
4,7KLHPSVVE87D69B,108,[],"[120, 110]"
